## Vector search with Gemini Embeddings

<a target="_blank" href="https://colab.research.google.com/github/weaviate/recipes/blob/main/weaviate-features/bring-your-own-vectors-and-multi-lingual/vector_search_gemini_embeddings.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This tutorial shows you, how to use the generally available (GA) Gemini Embedding model `gemini-embedding-001` in Weaviate.

The Gemini Embedding model is initialised from Google's Gemini LLM. It builds its representations on top of the knowledge already present in Gemini's parameters.

Model overview:

- Free and paid ($0.15/1M tokens) tiers available
- Multilingual capabilities in over 100 languages
- Leverages Matryoshka Representation Learning for flexible output dimensions (768, 1536, 3,072)
- Maximum input sequence length of 2,048 tokens

Note, if you have been using the experimental `gemini-embedding-exp-03-07`, you won't need to re-embed your contents according to the release blog, but it will no longer be supported by the Gemini API on August 14, 2025.

For more information, you can check out the following resources:

- [Release blog of GA version](https://developers.googleblog.com/en/gemini-embedding-available-gemini-api/)
- [Release blog of experimental version](https://developers.googleblog.com/en/gemini-embedding-text-model-now-available-gemini-api/)
- [Paper: Gemini Embedding: Generalizable Embeddings from Gemini](https://arxiv.org/pdf/2503.07891)

You can access the `gemini-embedding-001` model directly in Weaviate by using the [Google module](https://docs.weaviate.io/weaviate/model-providers/google/embeddings). Please note: you will need to use Weaviate  1.32.1 or later.

## Dependencies

In [1]:
%%capture
%pip install -q -U "google-genai>=1.0.0"
%pip install -q -U weaviate-client

## Connect to Weaviate

You can connect to Weaviate in different ways.

- [Weaviate Cloud (WCD)](https://console.weaviate.cloud/): Managed services for development and production environments.
- Embedded Weaviate: Experimental in-memory option without API keys for quick experimentation.
- Local Weaviate instance e.g. deployed with Docker

You can get 14 days of free access to Weaviate Cloud's Sandbox by creating an account in [Weaviate Cloud (WCD)](https://console.weaviate.cloud/), where you can also obtain your API key. No name, no credit card required.

Alternatively, you can enable option 2 for quick experimentation.

In [3]:
import weaviate
from weaviate.classes.init import Auth
import os

# Best practice: store your credentials in environment variables
weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

# Option 1: Connect to your Weaviate Cloud Service cluster
weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

# Option 2: Connect to an embedded instance
# client = weaviate.connect_to_embedded()

# Option 3: Connect to your local Weaviate instance deployed with Docker
# client = weaviate.connect_to_local()

weaviate_client.is_ready()

INFO:weaviate-client:Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.30.5/weaviate-v1.30.5-Linux-amd64.tar.gz
INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 752


True

## Create a collection

> Collection stores your data and vector embeddings.


In [4]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "MyCollection", and then you need to re-import it again.
import weaviate.classes.config as wc

# Delete the collection if it already exists
if (weaviate_client.collections.exists("MyCollection")):
    weaviate_client.collections.delete("MyCollection")

# Create a collection
collection = weaviate_client.collections.create(
    "MyCollection",
    vectorizer_config=wc.Configure.Vectorizer.none(),
    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Text", data_type=wc.DataType.TEXT),
    ]
)

print("Successfully created collection: MyCollection.")

Successfully created collection: MyCollection.


## Import the data

In [5]:
from google import genai

GOOGLE_APIKEY = os.environ["GOOGLE_AI_STUDIO_API_KEY"]

# Initialize the client
google_client = genai.Client(api_key=GOOGLE_APIKEY)


# Set up our multi-lingual data
multilingual_data = [
    {"text": "The quick brown fox jumps over the lazy dog."},
    {"text": "Der schnelle braune Fuchs springt über den faulen Hund."},

    {"text": "Artificial intelligence is transforming industries."},
    {"text": "Künstliche Intelligenz verändert Industrien."},

    {"text": "I love to read books in my free time."},
    {"text": "Ich lese gerne Bücher in meiner Freizeit."},

    {"text": "Beautiful weather today, perfect for a walk."},
    {"text": "Schönes Wetter heute, perfekt für einen Spaziergang."},

    {"text": "The capital of Germany is Berlin."},
    {"text": "Die Hauptstadt Deutschlands ist Berlin."}

]

# Get a list of texts to embed
texts = [item["text"] for item in multilingual_data]

# Generate embeddings with Gemini
result = google_client.models.embed_content(
    model="gemini-embedding-001",
    contents=texts
)

# Place embeddings in a list
embeddings = [embedding.values for embedding in result.embeddings]

# Print info about embeddings
print(f"Number of embeddings: {len(embeddings)}")
print(f"Length of each embedding: {len(embeddings[0])}")

# Add embeddings to multilingual dataset
for i, item in enumerate(multilingual_data):
    item["embedding"] = embeddings[i]

Number of embeddings: 10
Length of each embedding: 3072


In [6]:
import weaviate.classes as wvc

objs_to_insert = list()

for i, d in enumerate(multilingual_data):
    objs_to_insert.append(wvc.data.DataObject(
        properties={
            "text": d["text"],
        },
        vector=d["embedding"]
    ))


my_collection = weaviate_client.collections.get("MultilingualData")
my_collection.data.insert_many(objs_to_insert)

print("Data import complete")

Data import complete


Quick check to see if all objects are in.

In [7]:
len(my_collection)

10

## Query Weaviate: Vector Search (vector embeddings)

Search Weaviate with a vector embedding with [`near_vector`](https://docs.weaviate.io/weaviate/search/similarity#search-with-a-vector).

See [the docs](https://docs.weaviate.io/weaviate/search/similarity#search-with-a-vector) for more.

In [8]:
query = "What's the capital of Germany?"

query_embedding = google_client.models.embed_content(
    model="gemini-embedding-001",
    contents=query,
)

response = my_collection.query.near_vector(
    near_vector=query_embedding.embeddings[0].values,
    include_vector=True,
    limit=3
)

for item in response.objects:
  #print(item.uuid)
  print(item.properties['text'])
  print(item.vector, "\n")

The capital of Germany is Berlin.
{'default': [-0.02516309916973114, 0.014082825742661953, 0.01286312472075224, -0.06086959317326546, -0.03566242381930351, 0.005528680048882961, -0.005698827560991049, -0.01167087908834219, 0.019082311540842056, -0.00633844081312418, -0.026298299431800842, 0.0027747827116400003, 0.012228330597281456, 0.023666365072131157, 0.09661281108856201, 0.02636878564953804, -0.012430659495294094, -0.00392215745523572, -0.03805816173553467, -0.0319707952439785, -0.03071528673171997, -0.003802082035690546, 0.02905232273042202, -0.0015445764875039458, -0.016300339251756668, 0.008243197575211525, 0.008705693297088146, 0.0019047564128413796, 0.0011980924755334854, 0.024127313867211342, 0.016773568466305733, 0.012801848351955414, 0.010036628693342209, -0.01585620641708374, 0.02301114611327648, -0.002198915695771575, 0.010354913771152496, -0.0011373857269063592, 0.0029868006240576506, 0.0004731460940092802, -0.02656933292746544, 0.0022608700674027205, -0.0068858447484672